<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<h1 style="text-align:center;">Python: Objektorientierung</h1>
<h2 style="text-align:center;">Coding Akademie München GmbH</h2>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<div style="text-align:center;">Allaithy Raed</div>

# Objektorientierung

- Wir haben im vorherigen Kapitel Klassen kennengelernt, einen der grundlegenden Baustein der objektorientierten Programmierung
- In diesem Kapitel werden wir Vererbung betrachten.

## Attribute von Klassen

In [ ]:
class CountedAdder:
    num_counters = 0
    
    def __init__(self, value):
        CountedAdder.num_counters += 1
        self.value = value
    
    def describe(self):
        print(f"One of {CountedAdder.num_counters} adders. "
              f"This one adds {self.value} to its argument.")
    
    def add(self, n):
        return self.value + n

In [ ]:
print(CountedAdder.num_counters)
a1 = CountedAdder(10)
print(CountedAdder.num_counters)
a2 = CountedAdder(20)
print(CountedAdder.num_counters)

In [ ]:
print(a1.add(1))
print(a2.add(2))

In [ ]:
a1.describe()
a2.describe()

In [ ]:
print(CountedAdder.num_counters)
print(a1.num_counters)
print(a2.num_counters)

In [ ]:
print(CountedAdder.add)
print(a1.add)
print(a2.add)

## Vererbung

In [ ]:
class LoggingAdder(CountedAdder):
    def add(self, n):
        print(f"Adding {self.value} to {n}")
        return self.value + n

In [ ]:
a3 = LoggingAdder(30)
print(a3.add(3))
print(a3.num_counters)

In [ ]:
a1.describe()
a2.describe()
a3.describe()

In [ ]:
# Method Resolution Order:
LoggingAdder.mro()

In [ ]:
print(CountedAdder.add)
print(a1.add)
print(a2.add)
print(LoggingAdder.add)
print(a3.add)

In [ ]:
print(CountedAdder.add)
print(a1.add.__func__)
print(a2.add.__func__)
print(LoggingAdder.add)
print(a3.add.__func__)

## Abstrakte Klassen

- Die Klasse `ABC`

# Workshop

Siehe `070x-Workshop RPG-Würfel` bis `Factory für RPG-Würfel`.

# Reguläre Ausdrücke

# Workshop

Siehe `070x-Workshop RPG-Würfel` bis `Factory für RPG-Würfel`.

# Workshop

Siehe Verzeichnis `Examples/NameGenerator`.